<a href="https://colab.research.google.com/github/rickielin/Cardiac_Segmentation/blob/main/Cardiac_3_Domain_translation_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DICOM --> JPEG <br>
for CycleGAN / CUT training!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pydicom SimpleITK dicom2nifti

import numpy as np
import pandas as pd
from numpy import *
import seaborn as sns
import scipy.io as scio
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
import pydicom
import random
import glob
from tqdm import tqdm
import SimpleITK as sitk
import cv2
import nibabel as nib
import dicom2nifti
from skimage.transform import resize
%matplotlib inline

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 77.1 MB/s eta 0:00:00


In [2]:
DATASET_FOLDER = "/content/drive/MyDrive/Temporary/Cine_LGE_samples"
# 5 example cases only

patient_case_num = []
for i in os.listdir(DATASET_FOLDER):
    patient_case_num.append(i)

patient_case_num.remove('Matching_frame_number_in_Cine_images.xlsx')
patient_case_num

['4', '49', '26', '83', '310']

### train A: LGE

In [7]:
!mkdir /content/LGE_JPEG_FOLDER

In [ ]:
# Path to store JPEG files
LGE_JPEG = '/content/LGE_JPEG_FOLDER'

for case_num in patient_case_num:
    # 获取 _n 的个数
    Infarct_mask = scio.loadmat(DATASET_FOLDER+f"/{case_num}/Masks/InfarctMask.mat")
    Infarct_array = Infarct_mask['InfarctMask']
    mri_count = Infarct_array.shape[2]

    for _n in range(mri_count):
        lge_processing = f'{DATASET_FOLDER}/{case_num}/{case_num}_{_n+1}/LGE'
        for i in os.listdir(lge_processing):
            # IMG-0006-00012.dcm
            if "dcm" in i:
                lge_now = lge_processing + "/" + i

        print(lge_now)

        dicom_file = lge_now
        jpeg_file = f'{LGE_JPEG}/{case_num}_{_n+1}_lge.jpeg'  # output path

        dcm = pydicom.dcmread(dicom_file)  # 读取 DICOM 文件
        pixel_array = dcm.pixel_array  # 获取像素数据
        pixel_array_3d = pixel_array[:,:,np.newaxis]  # 把2D数组转换为3D
        resized_pixel_array = resize(pixel_array_3d, (512, 512, 1))  # 改变图像大小到(512, 512, 1)
        flipped_pixel_array = np.flip(resized_pixel_array, axis=0)  # 上下翻折图像
        rotated_pixel_array = np.rot90(flipped_pixel_array, 3, (0, 1))  # 逆时针旋转270度

        # 使用matplotlib保存JPEG
        plt.imsave(jpeg_file, rotated_pixel_array[:,:,0], cmap='gray')

        # 或者使用PIL保存JPEG
        # im = Image.fromarray((rotated_pixel_array[:,:,0] * 255).astype(np.uint8))
        # im.save(jpeg_file)

### train B: Cine

In [3]:
# 当前实验，仅抽取Cine中与LGE匹配的slices加入trainB。
# 不确定是否需要以此方式去除data imbalance。

In [4]:
# Matching_frame_number_in_Cine_images - TABLE

df_match = pd.read_excel('/content/drive/MyDrive/Temporary/Cine_LGE_samples/Matching_frame_number_in_Cine_images.xlsx')
print(df_match.head())

   CaseID  Matching frame number in Cine images
0       4                                    22
1       6                                    22
2      17                                    20
3      18                                    21
4      21                                    22


In [5]:
!mkdir /content/CINE_JPEG_FOLDER

In [8]:
# Path to store JPEG files
CINE_JPEG = '/content/CINE_JPEG_FOLDER'

for case_num in patient_case_num:
    # 获取 _n 的个数
    Infarct_mask = scio.loadmat(DATASET_FOLDER+f"/{case_num}/Masks/InfarctMask.mat")
    Infarct_array = Infarct_mask['InfarctMask']
    mri_count = Infarct_array.shape[2]

    for _n in range(mri_count):
        cine_processing = f'{DATASET_FOLDER}/{case_num}/{case_num}_{_n+1}/Cine_new'
        files = glob.glob(cine_processing + '/*')
        files = sorted(files)  # 对文件名进行排序
        match_num = df_match[df_match['CaseID']==int(case_num)]['Matching frame number in Cine images'].values[0] -1
        print(f"processing: {case_num}_{_n+1}, match number == {match_num + 1}")

        cine_processing = f'{files[match_num]}'
        # print(cine_processing)

        dicom_file = cine_processing
        jpeg_file = f'{CINE_JPEG}/{case_num}_{_n+1}_cine.jpeg'  # output path

        dcm = pydicom.dcmread(dicom_file)  # 读取 DICOM 文件
        pixel_array = dcm.pixel_array  # 获取像素数据
        pixel_array_3d = pixel_array[:,:,np.newaxis]  # 把2D数组转换为3D
        resized_pixel_array = resize(pixel_array_3d, (512, 512, 1))  # 改变图像大小到(512, 512, 1)
        flipped_pixel_array = np.flip(resized_pixel_array, axis=0)  # 上下翻折图像
        rotated_pixel_array = np.rot90(flipped_pixel_array, 3, (0, 1))  # 逆时针旋转270度

        # 使用matplotlib保存JPEG
        plt.imsave(jpeg_file, rotated_pixel_array[:,:,0], cmap='gray')

        # 或者使用PIL保存JPEG
        # im = Image.fromarray((rotated_pixel_array[:,:,0] * 255).astype(np.uint8))
        # im.save(jpeg_file)

processing: 4_1, match number == 22
processing: 4_2, match number == 22
processing: 4_3, match number == 22
processing: 49_1, match number == 19
processing: 49_2, match number == 19
processing: 49_3, match number == 19
processing: 49_4, match number == 19
processing: 49_5, match number == 19
processing: 26_1, match number == 19
processing: 26_2, match number == 19
processing: 83_1, match number == 23
processing: 83_2, match number == 23
processing: 83_3, match number == 23
processing: 83_4, match number == 23
processing: 83_5, match number == 23
processing: 310_1, match number == 24
processing: 310_2, match number == 24
